In [1]:
import re
import logging
from abc import ABC, abstractmethod

In [2]:
class SynonymsFinder(ABC):
    
    @abstractmethod
    def find_all(text): pass

In [3]:
class TitlesInParenthesesFinder(SynonymsFinder):
    """Example:
        <word> ('''<synonym>''', '''<synonym2>''' - ...)
    """
    TEXT_IN_PARENTHESES_PATTERN = re.compile(r"\((.*)\)")
    TITLES_PATTERN = re.compile(r"'''(.+?)'''")

    def find_all(self, text):
        parens_match = self.TEXT_IN_PARENTHESES_PATTERN.search(text)
        if parens_match is None:
            return
        text_in_parens = parens_match.group(1)
        for match in self.TITLES_PATTERN.finditer(text_in_parens):
            yield match.group(1)

In [4]:
class TitleAfterLiteralFinder(SynonymsFinder):
    """Example:
        <word>, lub '''<synonym>''' - ...
    """

    def __init__(self, literal):
        self.pattern = re.compile(fr"{literal}\s+'''(.+?)'''")

    def find_all(self, text):
        match = self.pattern.search(text)
        if match is None:
            return ()
        return (match.group(1),)

In [5]:
class WordAfterLiteralFinder(SynonymsFinder):
    """This differs from the above one
    in that the word searched for is not escaped by "'''",
    so there can only one, no spaces. Example:
        <word> - inaczej <synonym> - ...
    """

    def __init__(self, literal):
        self.pattern = re.compile(fr"{literal}\s+(\w+)")
    
    def find_all(self, text):
        match = self.pattern.search(text)
        if match is None:
            return ()
        return (match.group(1),)

In [6]:
class ArticleProcesser:
    BASE_WORD_PATTERN = re.compile(r"'''(.+?)'''")

    class CannotFindBaseWord(Exception):
        
        def __init__(self, text):
            self.text = text
        
        def __str__(self):
            return "Could not find base word in: %r" % self.text[:50]
    
    def __init__(self, synonyms_finders):
        self.synonyms_finders = synonyms_finders

    def find_base_word(self, text):
        match = self.BASE_WORD_PATTERN.search(text)
        if match is None:
            raise self.CannotFindBaseWord(text)
        return match.group(1), match.end()

    def find_synonyms(self, text):
        try:
            base_word, end = self.find_base_word(text)
            text = text[end:]
        except self.CannotFindBaseWord as e:
            logging.error(e)
            return ()
        words = [base_word]
        for finder in self.synonyms_finders:
            synonyms = finder.find_all(text)
            words.extend(synonyms)
        unique = tuple(dict.fromkeys(words))
        if len(unique) == 1:
            return ()
        return unique

In [7]:
def stream_articles_texts(source):
    lines = iter(source)
    try:
        while True:
            _title = next(lines)
            text = next(lines)
            _blank_line = next(lines)
            yield text
    except StopIteration:
        return

In [8]:
ARTICLES_FILE_PATH = 'data/poczatki_wikipediowe.txt'

processer = ArticleProcesser((
    TitleAfterLiteralFinder('lub'),
    TitleAfterLiteralFinder(','),
    WordAfterLiteralFinder('inaczej'),
    TitlesInParenthesesFinder(),
))
    
with open(ARTICLES_FILE_PATH) as f:
    for art, _ in zip(stream_articles_texts(f), range(10_000)):
        if syns := processer.find_synonyms(art):
            print(' # '.join(w.lower() for w in syns))

ERROR:root:Could not find base word in: 'Torquay |data i miejsce śmierci = 12 stycznia 1976'
ERROR:root:Could not find base word in: '|powierzchnia_wód = 1350 km² (4,7%) |powierzchnia_'
ERROR:root:Could not find base word in: '|gęstość = 99,5 |gęstość_miejsce = 78 |ludność_rok'
ERROR:root:Could not find base word in: '|powierzchnia_wód = ~0% |powierzchnia_miejsce = 17'
ERROR:root:Could not find base word in: '* AMP - wieloprocesorowość asynchroniczna: archite'
ERROR:root:Could not find base word in: '* Aleksander Orłowski - malarz koni i scen batalis'
ERROR:root:Could not find base word in: '* Arizona - gatunek węży * Arizona - stan w USA * '
ERROR:root:Could not find base word in: 'border | pieczęćgrafika = Seal of Alabama.svg | ma'
ERROR:root:Could not find base word in: 'Ulm, Cesarstwo Niemieckie | data_smierci = 18 kwie'
ERROR:root:Could not find base word in: '* Alba - imię żeńskie * Alba - w jęz. irlandzkim i'
ERROR:root:Could not find base word in: '* Alfa Centauri - gwiazda * A

aksjomat # pewnik # postulat
alternatywa # suma logiczna
przylądek igielny # przylądek agulhas
antropologia # λόγος # άνθρωπος
aceton # propanon
astma oskrzelowa # dychawica oskrzelowa
polonezköy # adampol
aix # ''a # i # x
1 # 4
argon # ar
amonity # amonitowate
alfabet hebrajski # pismem kwadratowym
anime # anime
alergia # nadwrażliwość # uczulenie
argyll # argylshire
aidan # aedan
alfabet łaciński # łacinka
ajnowie # ajnosi # ajnu # ajnos
abu bakr # abū bakr
andreas vesalius # wesaliusz
arpanet # ''a # r # p # a # net
armia krajowa # ak
aszkenazyjczycy #  אַשְׁכְּנָזִים
aleksander isajewicz sołżenicyn # aleksander isaakowicz
amfetamina # 1-fenylo-2-aminopropan
anime music video # amv
aleksjan bassianus # aleksander sewer
allah # ﷲ
astma # dychawica
abu nidal # hassan sabri al banna
anastazjusz # dikoros
alfabet hawajski # e
achemenidzi # هخامنشیان
ariowie # indo-irańczycy
akwaforta # kwasoryt
antygen # anti # gen
andrzej frycz modrzewski # andreas fricius modrevius
angloameryka # ame

ERROR:root:Could not find base word in: '* ban - jednostka informacji * ban - dawna drobna '
ERROR:root:Could not find base word in: '\n'
ERROR:root:Could not find base word in: '\n'
ERROR:root:Could not find base word in: '\n'
ERROR:root:Could not find base word in: '(Reykjavík) |pochodzenie = |data_śmierci = |miejsc'
ERROR:root:Could not find base word in: '| producent = | rodzaj = pistolet maszynowy| !-- n'
ERROR:root:Could not find base word in: '- miniatura krajobrazu (góry, morza, jeziora, wodo'
ERROR:root:Could not find base word in: '* Biwa (instrument muzyczny) pochodzący z wschodni'
ERROR:root:Could not find base word in: 'Austin |siedziba = |nr KRS = |prezes = |ważni prac'
ERROR:root:Could not find base word in: '* Bezpartyjny Blok Współpracy z Rządem Józefa Piłs'
ERROR:root:Could not find base word in: '/> # REDIRECT\n'
ERROR:root:Could not find base word in: '\n'
ERROR:root:Could not find base word in: '\n'
ERROR:root:Could not find base word in: '/> # REDIRECT Ludwig van\

białka adhezyjne # adhezyny
boże narodzenie # uroczystość narodzenia pańskiego
bekerel # bq
borys godunow # борис годунов # борисъ годуновъ
borowski # borowska # borowscy
czechizm # bohemizm
bitwa pod grunwaldem # pierwsza bitwa pod tannenbergiem
bizmut # bi
biotyna # witamina h
cpan # ''c # p # a # n
chrześcijaństwo # chrystianizm
chlor # cl
chrom # cr
cynk # zn
cd-audio # audio-cd
charles george gordon # gordon pasza
clips # c language integrated production system
ciało skończone # ciało galois
centymetr # cm
choroby zakaźne # choroby infekcyjne
cmyk # kolory triadowe # barwy procesowe
cytadela warszawska # cytadela aleksandrowska
czarna afryka # afryka subsaharyjska
chasydyzm # חסידות
cadyk, cadik, sadyk # צדיק
chmielaki krasnostawskie # dożynkami
cent # eurocent
poznański czerwiec 1956 # czerwiec ’56
così fan tutte # fiordiligi
czasław # czesław
charon # charun
chlorek sodu # nacl
choroba heinego-medina # wirusowe zapalenie rogów przednich rdzenia kręgowego # h14
ceres # cerera
dob

ERROR:root:Could not find base word in: '/> # REDIRECT\n'
ERROR:root:Could not find base word in: '\n'
ERROR:root:Could not find base word in: '/> # REDIRECT\n'
ERROR:root:Could not find base word in: '/> # REDIRECT\n'
ERROR:root:Could not find base word in: '/> # REDIRECT\n'
ERROR:root:Could not find base word in: '/> # REDIRECT\n'
ERROR:root:Could not find base word in: '* drachma (jednostka masy) * drachma (jednostka ob'
ERROR:root:Could not find base word in: '/> # REDIRECT\n'
ERROR:root:Could not find base word in: '\n'
ERROR:root:Could not find base word in: '|Data śmierci = |Miejsce śmierci = |Zawód = aktor '
ERROR:root:Could not find base word in: '(Basildon) |gatunek = nowa fala ref name = www / b'
ERROR:root:Could not find base word in: '/> # REDIRECT fale\n'
ERROR:root:Could not find base word in: '/> # REDIRECT\n'
ERROR:root:Could not find base word in: '\n'
ERROR:root:Could not find base word in: '/> # REDIRECT Didiusz\n'
ERROR:root:Could not find base word in: '\n'
ERROR:

...

ERROR:root:Could not find base word in: '* szereg (matematyka) * szereg (wojsko) * szereg\n'
ERROR:root:Could not find base word in: '\n'
ERROR:root:Could not find base word in: '* czop – w budowie maszyn, część osi lub wału, na '
ERROR:root:Could not find base word in: 'Niżnyj Alkiejew |Data śmierci = |Miejsce śmierci ='
ERROR:root:Could not find base word in: '* Rycerze Okrągłego Stołu * Okrągły Stół (legendy)'
ERROR:root:Could not find base word in: '| producent = Renault| typ = czołg lekki| !-- np. '
ERROR:root:Could not find base word in: '\n'
ERROR:root:Could not find base word in: 'charakterystyka pompy wyporowej Pawel Wajda,\n'
ERROR:root:Could not find base word in: '\n'
ERROR:root:Could not find base word in: '* CAE - certyfikat znajomości języka angielskiego '
ERROR:root:Could not find base word in: '\n'
ERROR:root:Could not find base word in: '* ciężki karabin\n'
ERROR:root:Could not find base word in: '* Absalon, inaczej Absalom - imię męskie * Absalon'
ERROR:root:Could no

ERROR:root:Could not find base word in: 'Pawel Wajda,\n'
ERROR:root:Could not find base word in: '\n'
ERROR:root:Could not find base word in: '18. ceremonia rozdania Oscarów odbyła się 7 marca '
ERROR:root:Could not find base word in: '\n'
ERROR:root:Could not find base word in: '19. ceremonia rozdania Oscarów odbyła się 13 marca'
ERROR:root:Could not find base word in: '| producent = | projektant = | rodzaj = karabin au'
ERROR:root:Could not find base word in: 'Pawel Wajda,\n'
ERROR:root:Could not find base word in: 'Pawel Wajda,\n'
ERROR:root:Could not find base word in: '* Kalliope – jedna z muz w mitologii greckiej * Ka'
ERROR:root:Could not find base word in: '* Allegro.pl (serwis internetowy) &amp;ndash; pols'
ERROR:root:Could not find base word in: 'Pawel Wajda,\n'
ERROR:root:Could not find base word in: '\n'
ERROR:root:Could not find base word in: '\n'
ERROR:root:Could not find base word in: '20. ceremonia rozdania Oscarów odbyła się 20 marca'
ERROR:root:Could not find base wor

pneumatyczny przenośnik cieczy # powietrzny przenośnik cieczy
wenedowie # wendowie
alemanowie # alamanowie
kankan # cancan
uniwersytet harvarda # harvard university # harvard college
domenico da piacenza # domenico da ferrara
don juan # don juan czyli kamienny gość
dziadek do orzechów # щелкунчик # casse-noisette # the nutcracker
wilhelm i zdobywca # wilhelmem bękartem
fokstrot # foxtrot
autobracketing # aeb
kwazar # ''quas # ar # ''q # s # o
inżynier # inż.
bazylika grobu świętego # bazylika pańskiego grobu # bazylika bożego grobu
pompa przewodowa # pompa perystaltyczna
ata # ide # pata
sumer # שִׁנְעָר
ogólna teoria względności  # otw
partia pracy # laburzyści # komitet przedstawicielstwa robotniczego
pompa wirowa krętna # n # q # h
karnawał # zapusty
saliowie # salii
patrylokalność # patrylokalizm
matrylokalność # matrylokalizm
lokalizacja małżeństwa # małżeńskim lokalizmie # o
terpsychora # terpsichore
trójkątny kapelusz # trójgraniasty kapelusz
melpomene # melpomena
teoria modeli 

ERROR:root:Could not find base word in: '\n'
ERROR:root:Could not find base word in: '\n'
ERROR:root:Could not find base word in: '| data_śmierci = | miejsce_śmierci = | instrument '
ERROR:root:Could not find base word in: '41. ceremonia rozdania Oscarów odbyła się 14 kwiet'
ERROR:root:Could not find base word in: '\n'
ERROR:root:Could not find base word in: '\n'
ERROR:root:Could not find base word in: '* biga (poligrafia) * biga (wóz) * Biga (Turcja) -'
ERROR:root:Could not find base word in: '42. ceremonia rozdania Oscarów odbyła się 7 kwietn'
ERROR:root:Could not find base word in: '\n'
ERROR:root:Could not find base word in: '* Aspazja\n'
ERROR:root:Could not find base word in: '\n'
ERROR:root:Could not find base word in: '44. ceremonia rozdania Oscarów odbyła się 10 kwiet'
ERROR:root:Could not find base word in: '\n'


silnik widlasty # v
quake c # qc
